## boilerplate code

In [1]:
import supy as sp
import pandas as pd
import numpy as np
from pathlib import Path
p_dir_input = Path("../data/canyon_gbg")

## load in observation data

In [4]:
# load in observation data
p_obs = p_dir_input / "GbgTorgg_canyon_QSresid.csv"
df_obs_raw = pd.read_csv(p_obs, sep=";")

# convert to datetime
ser_year = pd.to_datetime(df_obs_raw.year.astype(str) + "-01-01")
# df_obs_raw.dectime
ser_dt=ser_year+pd.to_timedelta(df_obs_raw.dectime, unit="D").dt.round("30T")
df_obs=df_obs_raw.copy()
df_obs["datetime"]=ser_dt
df_obs.set_index("datetime", inplace=True)
df_obs.drop(columns=["year", "dectime"], inplace=True)
ser_qg = df_obs.dryQSresid.rename("QG_obs")
ser_qg

datetime
2004-01-04 01:00:00   -64.5645
2004-01-04 01:30:00   -58.4494
2004-01-04 02:00:00   -50.1401
2004-01-04 02:30:00   -40.1201
2004-01-04 03:00:00   -35.8317
                        ...   
2004-08-24 19:00:00        NaN
2004-08-24 21:00:00        NaN
2004-08-24 20:00:00        NaN
2004-08-24 22:00:00        NaN
2004-08-24 21:00:00        NaN
Name: QG_obs, Length: 11228, dtype: float64

## simulations

In [9]:

p_runcontrol = p_dir_input / "RunControl.nml"
df_state_base = sp.init_supy(p_runcontrol)

df_forcing = sp.load_forcing_grid(p_runcontrol, 1)

2024-04-15 16:49:13,097 - SuPy - INFO - All cache cleared.


ValueError: invalid literal for int() with base 10: '('

In [10]:
# OHM
df_output_base, df_state_final = sp.run_supy(df_forcing, df_state_base)

NameError: name 'df_forcing' is not defined

In [18]:
# EHC as heat storage scheme
df_state_ehc = df_state_base.copy()
df_state_ehc.storageheatmethod = 5

# modify the depth of the vertical layers
ar_depth = np.repeat(np.array([[0.1, 0.1, 0.5, 2, 3]]), 7, axis=0).flatten()
df_state_ehc.loc[:, "dz_surf"] = ar_depth

# # modify the heat capacity of the vertical layers
# ar_cp = np.repeat(np.array([[2.2, 2.2, 2.2, 2.2, 2.2]])*1e6, 7, axis=0).flatten()
# df_state_ehc.loc[:, "cp_surf"] = ar_cp

# # modify the thermal conductivity of the vertical layers
# ar_k = np.repeat(np.array([[1.1, 1.1, 1.1, 1.1, 1]]) * 1.5, 7, axis=0).flatten()
# df_state_ehc.loc[:, "k_surf"] = ar_k

df_output_ehc, df_state_final = sp.run_supy(df_forcing, df_state_ehc)

NameError: name 'df_state_base' is not defined

In [11]:
df_state_ehc.loc[:,"k_surf"]


NameError: name 'df_state_ehc' is not defined

### result comparison

In [12]:
df_comp = (
    pd.concat([df_output_base, df_output_ehc], axis=1, keys=["base", "ehc"])
    .swaplevel(axis=1, i=1, j=0)
    .swaplevel(axis=1, i=1, j=2)
    .sort_index(axis=1)
)
df_comp_qs = df_comp.loc[1, "SUEWS"].QS
df_comp_qs = df_comp_qs.assign(obs=ser_qg).dropna()
df_comp_qs
df_comp_qs.loc['2004-2-13':'2004-2-19'].plot()

NameError: name 'df_output_base' is not defined

In [13]:
ax = df_comp_qs.plot.scatter(x="obs", y="ehc", alpha=0.2, style="..", title="EHC")
_ = ax.set_aspect("equal", "box")
_ = ax.set_xlim(-200, 400)
_ = ax.set_ylim(-200, 400)
_ = ax.plot([-200, 400], [-200, 400], "k--")

NameError: name 'df_comp_qs' is not defined

In [14]:
ax = df_comp_qs.plot.scatter(x="obs", y="base", alpha=0.2, style="..", title="OHM")
_ = ax.set_aspect("equal", "box")
_ = ax.set_xlim(-200, 400)
_ = ax.set_ylim(-200, 400)
_ = ax.plot([-200, 400], [-200, 400], "k--")

NameError: name 'df_comp_qs' is not defined

In [15]:
# compare the performance of the two simulations
df_comp_qs.corr()


NameError: name 'df_comp_qs' is not defined

In [16]:
df_comp_qs.corr(method="spearman")

NameError: name 'df_comp_qs' is not defined

In [17]:
# calculate RMSE for the two simulations
rmse_base = np.sqrt(((df_comp_qs.obs - df_comp_qs.base) ** 2).mean())
rmse_ehc = np.sqrt(((df_comp_qs.obs - df_comp_qs.ehc) ** 2).mean())
rmse_base, rmse_ehc


NameError: name 'df_comp_qs' is not defined